# ✅ 店長のための次アクション生成 v6.1
- 指定CSVの当日（データ最大日）に基づき、フラグ×カテゴリの上振れから実行ToDoを出力
- 在庫・原価なし前提。面出し/フェース/前出し/時間帯強化にフォーカス

In [ ]:
# 日本語フォント設定
import font_setup


In [ ]:
import warnings; warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
from pathlib import Path
import os
try:
    import ipywidgets as widgets; from IPython.display import display, clear_output
    WIDGETS=True
except Exception:
    WIDGETS=False

def pick_enriched_csv():
    for env in ('DATA_PATH','ENRICHED_CSV'):
        p=os.environ.get(env)
        if p and Path(p).exists(): return Path(p)
    out=Path('output')
    if out.exists():
        cand=sorted(out.glob('06_final_enriched_*.csv'), reverse=True)
        if cand: return cand[0]
    fixed=Path('output/06_final_enriched_20250701_20250930.csv')
    if fixed.exists(): return fixed
    return None

DATA_PATH = pick_enriched_csv()
if not DATA_PATH:
    raise FileNotFoundError('06_final_enriched_*.csv が見つかりません。/home/rapids/work/output に配置、または環境変数 DATA_PATH を指定してください。')
df = pd.read_csv(DATA_PATH, encoding='utf-8-sig')
df = df.rename(columns={'店舗':'store_id','商品名':'sku_id','日付':'date','売上数量':'qty','売上金額':'sales_amt',
    'フェイスくくり大分類':'category_l','フェイスくくり中分類':'category_m','フェイスくくり小分類':'category_s'})
df['date'] = pd.to_datetime(df['date'])
df['qty'] = pd.to_numeric(df['qty'], errors='coerce').fillna(0)
df['sales_amt'] = pd.to_numeric(df['sales_amt'], errors='coerce').fillna(0)
stores_all = sorted(df['store_id'].unique())
stores_sel = stores_all[:1]
if WIDGETS:
    w = widgets.SelectMultiple(options=stores_all, value=tuple(stores_sel), description='店舗(複数)')
    display(w)
    stores_sel = list(w.value)
today = df['date'].max()
print('基準日:', today, '/ 対象店舗:', stores_sel)


## 提案ロジック（カテゴリuplift × 条件フラグ）

In [ ]:
flag_cols = [c for c in ['降雨フラグ','週末フラグ','猛暑日','真夏日','夏日','冬日','真冬日','給料日','給料日直後','月初3日','月末3日'] if c in df.columns]
sub = df[(df['date']==today) & (df['store_id'].isin(stores_sel))][['store_id']+flag_cols].drop_duplicates() if flag_cols else pd.DataFrame()
today_flags = {c:int(sub[c].max()) for c in flag_cols} if not sub.empty else {}
print('今日のフラグ:', {k:v for k,v in today_flags.items() if v==1})

def propose_by_flag(df_all, stores, flag_col, top_n=5):
    if flag_col not in df_all.columns: return pd.DataFrame()
    d = df_all[df_all['store_id'].isin(stores)].copy()
    grp_cols = ['date','store_id','category_l'] if 'category_l' in d.columns else ['date','store_id']
    base = d.groupby(grp_cols, as_index=False)['sales_amt'].sum()
    flags = d[grp_cols+[flag_col]].drop_duplicates()
    m = base.merge(flags, on=grp_cols, how='left')
    if 'category_l' not in m.columns: return pd.DataFrame()
    a = m[m[flag_col]==1].groupby('category_l', as_index=False)['sales_amt'].mean()
    b = m[m[flag_col]!=1].groupby('category_l', as_index=False)['sales_amt'].mean()
    if a.empty or b.empty: return pd.DataFrame()
    u = a.merge(b, on='category_l', suffixes=('_flag1','_flag0'))
    u['uplift'] = (u['sales_amt_flag1']-u['sales_amt_flag0'])/(u['sales_amt_flag0']+1e-6)
    return u.sort_values('uplift', ascending=False).head(top_n)

action_templates = {
    '降雨フラグ': ['ホットスナック/温かい総菜のフェース増', '入口付近に温かい食品を前出し', '傘/雨具の見切り品前出し(あれば)'],
    '週末フラグ': ['弁当/麺/デザートのフェース増', 'ピーク前(11時/17時)の前出し強化', 'レジ前クロスMD'],
    '猛暑日': ['冷飲料/アイスのフェース増', 'チルドデザートの前出し', '氷/保冷品(あれば)の導線強化'],
    '真夏日': ['冷飲料のフェース増', '麺(冷)の前出し'],
    '夏日': ['冷飲料/サラダの前出し'],
    '冬日': ['温かい総菜/カップ麺のフェース増'],
    '真冬日': ['ホット飲料/中華まん(あれば)の前出し'],
    '給料日': ['高単価弁当/スイーツ前出し', '夕方の陳列厚め'],
    '給料日直後': ['嗜好品/デザートフェース増', 'クロスMD強化'],
    '月初3日': ['主力定番の在庫面確保', '発注厚め(在庫運用があれば)'],
    '月末3日': ['値頃品・節約訴求の前出し']
}

plans = []
for f,val in today_flags.items():
    if val!=1: continue
    u = propose_by_flag(df, stores_sel, f, 5)
    cats = list(u['category_l'].head(3)) if not u.empty else []
    plans.append({'flag': f, 'top_categories': cats, 'actions': action_templates.get(f, [])})

plans_df = pd.DataFrame(plans) if plans else pd.DataFrame(columns=['flag','top_categories','actions'])
display(plans_df)
if not plans_df.empty:
    print('
--- 実行ToDo(例) ---')
    for _,p in plans_df.iterrows():
        cats = ' / '.join(p['top_categories']) if p['top_categories'] else '主要カテゴリ'
        print(f"[条件: {p['flag']}] {cats}")
        for a in p['actions']:
            print('  -', a)
    print('
タイムライン例:')
    print('  開店前   : 前出し/フェース増（上記カテゴリ）')
    print('  昼ピーク前: 11時前の補充/前出し強化')
    print('  夕ピーク前: 17時前の補充/前出し強化')
    print('  閉店前   : 見切り/値頃訴求（在庫連動は未実装）')
else:
    print('今日の条件フラグが無い、または検出できません。標準運用を実施してください。')
